<a href="https://colab.research.google.com/github/6058ah/BusinessCapstone/blob/master/%EB%89%B4%EC%8A%A4%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
!pip install xlsxwriter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 검색시 리스트 크롤링하는 프로그램 > _select사용
- 크롤링 해오는 것 : 링크,제목,신문사,내용요약본
- 내용요약본  -> 정제 작업 필요
- 리스트 -> 딕셔너리 -> df -> 엑셀로 저장
'''''''''''''''''''''

#각 크롤링 결과 저장하기 위한 리스트 선언
title_text=[]
link_text=[]
source_text=[]
contents_text=[]
page_list=[]
result={}

#엑셀로 저장하기 위한 변수
RESULT_PATH ='/content/drive'  #결과 저장할 경로

#날짜 범위 리스트로 만들기
def date_range(start, end):
    start = datetime.strptime(start, "%Y.%m.%d")
    end = datetime.strptime(end, "%Y.%m.%d")
    dates = [date.strftime("%Y.%m.%d") for date in pandas.date_range(start, periods=(end-start).days+1)]
    return dates

#크롤링 시작
def crawler(maxpage,query,sort,s_date,e_date):
    dates = date_range(s_date, e_date)
    last = False
    page_num = 1
    maxpage_t =(int(maxpage)-1)*10+1 
    while page_num <= maxpage_t:
      for x in dates:
        y = x.replace(".","")
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + x + "&de=" + x + "&nso=so%3Ar%2Cp%3Afrom" + y + "to" + y + "%2Ca%3A&start=" + str(page_num)
        #url = "https://search.naver.com/search.naver?&where=news&query={0}&sm=tab_pge&sort={1}&photo=0&field=0&reporter_article=&pd=3&ds={2}&de={2}&docid=&nso=so:r,p:,a:all&mynews=1&cluster_rank=238&start={3}&refresh_start=0".format(query,sort,x,str(page_num))
        response = requests.get(url)
        html = response.text

        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')

        #<a>태그에서 제목과 링크주소 (a 태그 중 class 명이 news_tit인 것)
        atags = soup.find_all('a', 'news_tit')
        for atag in atags:
            title = atag.get('title')
            title_text.append(title)     #제목

        #모든 리스트 딕셔너리형태로 저장
        result= {"title":title_text}
        df = pd.DataFrame(result) #df로 변환
        title_text.clear()
        page_num += 10

    # 새로 만들 파일이름 지정
    outputFileName = '{0}~{1} {2} 관련 뉴스제목.xlsx'.format(s_date.replace(".",""), e_date.replace(".",""), query)
    writer = pd.ExcelWriter(outputFileName, engine='xlsxwriter')
    for i in range(len(dates)):
      df.to_excel(writer, sheet_name= dates[i])
    writer.save()

    #outputFileName = '{0}~{1} {2} 관련 뉴스제목.xlsx'.format(s_date.replace(".",""), e_date.replace(".",""), query)
    #df.to_excel(RESULT_PATH+outputFileName,sheet_name='sheet1')

#메인함수
def main():
    info_main = input("="*50+"\n"+"입력 형식에 맞게 입력해주세요."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    maxpage = input("최대 크롤링할 페이지 수 입력하시오: ") #10,20...
    query = input("검색어 입력: ") #네이버, 부동산...
    sort = input("뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): ")    #관련도순=0  최신순=1  오래된순=2
    s_date = input("시작날짜 입력:")
    e_date = input("끝날짜 입력:")
    crawler(maxpage,query,sort,s_date,e_date)

In [127]:
main()

입력 형식에 맞게 입력해주세요.
 시작하시려면 Enter를 눌러주세요.
최대 크롤링할 페이지 수 입력하시오: 10
검색어 입력: 2차전지
뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): 0
시작날짜 입력:2019.01.01
끝날짜 입력:2019.01.02


#참고
##https://hansuho113.tistory.com/3?category=913503
##https://adsp-ggini.tistory.com/80